# **Imports**

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier  # pip install xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# 1. Load Pre-trained Model (same as before)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

94773248/94765736 [==============================] - 0s 0us/step


In [3]:
def augment_brightness_contrast(image):
    alpha = 1.0 + random.uniform(-0.2, 0.2)
    beta = random.randint(-20, 20)
    new_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return new_image

def augment_color(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    s_factor = 1.0 + random.uniform(-0.2, 0.2)
    s = np.clip(s * s_factor, 0, 255).astype(np.uint8)
    hsv_img = cv2.merge((h, s, v))
    new_image = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return new_image

def augment_gamma(image):
    gamma = 1.0 + random.uniform(-0.3, 0.3)
    inv_gamma = 1.0 / gamma
    table = [((i / 255.0) ** inv_gamma) * 255 for i in range(256)]
    table = np.array(table).astype(np.uint8)
    new_image = cv2.LUT(image, table)
    return new_image

In [4]:
# 3. Data Generators (using flow_from_directory with augmentations)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=lambda x: x  # Placeholder for combined preprocessing
)

In [5]:
def combined_preprocessing(image):
    # 1. Point Processing Augmentations (applied to a *copy* of the original image)
    img_point = image.copy()  # Create a copy for point processing
    augmented_img_point = augment_brightness_contrast(img_point)
    augmented_img_point = augment_color(augmented_img_point)
    augmented_img_point = augment_gamma(augmented_img_point)
    img_array_point = cv2.resize(augmented_img_point, (224, 224)) # Resize after augmentation

    # 2. Convert to array and preprocess for ResNet
    img_array_point = image.img_to_array(img_array_point)
    img_array_point = np.expand_dims(img_array_point, axis=0)  # Add batch dimension
    img_array_point = tf.keras.applications.resnet50.preprocess_input(img_array_point)
    return img_array_point

In [10]:
def combined_preprocessing(image):
    # 1. Point Processing Augmentations (applied to a *copy* of the original image)
    img_point = image.copy()  # Create a copy for point processing
    augmented_img_point = augment_brightness_contrast(img_point)
    augmented_img_point = augment_color(augmented_img_point)
    augmented_img_point = augment_gamma(augmented_img_point)

    img_array_point = cv2.resize(augmented_img_point, (224, 224))  # Resize after augmentation

    # 2. Add batch dimension and preprocess for ResNet (NO img_to_array needed)
    img_array_point = np.expand_dims(img_array_point, axis=0)  # Add batch dimension
    img_array_point = tf.keras.applications.resnet50.preprocess_input(img_array_point)
    return img_array_point

In [11]:
trainroot='/kaggle/input/20-skin-diseases-dataset/Dataset/train'
testroot='/kaggle/input/20-skin-diseases-dataset/Dataset/test'
train_datagen.preprocessing_function = combined_preprocessing
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
train_generator = train_datagen.flow_from_directory(trainroot,target_size=(224, 224),batch_size=32,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(testroot,target_size=(224, 224),batch_size=32,class_mode='categorical')

Found 2609 images belonging to 19 classes.
Found 897 images belonging to 20 classes.


In [12]:
# 4. Feature Extraction (using generators)
def extract_features_from_generator(generator, steps):
    features = []
    labels = []
    for i in range(steps):
        x, y = generator.next()
        feature_batch = base_model.predict(x)
        features.extend(feature_batch)
        labels.extend(y)
    return np.array(features), np.array(labels)

In [19]:
def extract_features_from_generator(generator, steps):
    features = []
    labels = []
    for i in range(steps):
        x, y = generator.next()
        feature_batch = base_model.predict(x)
        features.extend(feature_batch)
        labels.extend(y)  # y is already in correct format from flow_from_directory
    return np.array(features), np.array(labels)

In [13]:
steps_train = len(train_generator)
print(steps_train)
steps_test = len(test_generator)
print(steps_test)

82
29


In [20]:
training_features, training_labels = extract_features_from_generator(train_generator, steps_train)
test_features, test_labels = extract_features_from_generator(test_generator, steps_test)

In [21]:
training_labels = np.array(training_labels).ravel()  # Or .flatten()
test_labels = np.array(test_labels).ravel()  # Or .flatten()

In [22]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # or other metrics
xgb_model.fit(training_features, training_labels) 

XGBoostError: [15:14:49] ../src/data/data.cc:583: Check failed: labels_.Size() == num_row_ (49571 vs. 2609) : Size of labels must equal to number of rows.
Stack trace:
  [bt] (0) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9133f) [0x7c9bc284633f]
  [bt] (1) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x110fcc) [0x7c9bc28c5fcc]
  [bt] (2) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1b90e7) [0x7c9bc296e0e7]
  [bt] (3) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1b99bc) [0x7c9bc296e9bc]
  [bt] (4) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x50) [0x7c9bc2835ed0]
  [bt] (5) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x69dd) [0x7c9c69d609dd]
  [bt] (6) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x6067) [0x7c9c69d60067]
  [bt] (7) /opt/conda/lib/python3.7/lib-dynload/_ctypes.cpython-37m-x86_64-linux-gnu.so(_ctypes_callproc+0x2f4)

In [23]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(training_features, training_labels)

ValueError: Found input variables with inconsistent numbers of samples: [2609, 49571]

In [ ]:
# 1. Load Data and Extract Features (using flow_from_directory)
# ... (Data generators and feature extraction using flow_from_directory as in previous responses)

# 2. Train XGBoost
 # Train on ALL training data

# 3. Train Random Forest
  # Train on ALL training data

# 4. Evaluate Models (on the test set)
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))

evaluate_model(xgb_model, test_features, test_labels, "XGBoost")
evaluate_model(rf_model, test_features, test_labels, "Random Forest")

# 5. Make Predictions (on new, unseen data)
def predict_new_image(model, image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    features = base_model.predict(img_array)
    new_features = features.flatten().reshape(1, -1)  # Reshape for prediction
    prediction = model.predict(new_features)
    return prediction

new_image_path = "path/to/new_image.jpg"
xgb_prediction = predict_new_image(xgb_model, new_image_path)
rf_prediction = predict_new_image(rf_model, new_image_path)

print("XGBoost Prediction:", xgb_prediction)
print("Random Forest Prediction:", rf_prediction)